# Library

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from gensim.models import FastText

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

from keras.utils.np_utils import to_categorical
from keras.models import load_model, Model, Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Embedding, SimpleRNN, Dropout

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Preprocessing
### Load Data

In [2]:
df = pd.read_pickle('preprocessed_df.pkl')
df = pd.DataFrame(df[['detokenize','plot']])
df.head()

,detokenize,plot
0,the desperate hour lakewood salah cerita suara...,1
1,edisi males review singkat tonton libur dp des...,1
2,plot utama orang deserter pursuit buru wamil j...,1
3,film hereditary horror thrill midsommar gatau ...,0
4,batman manusiawi tarung nya sadis scene pursui...,1


In [3]:
reviews = df['detokenize'].astype(str)
label = pd.get_dummies(df['plot']).values

# Variables

In [4]:
MAX_FEATURES = 10000
test_size_split = 0.1

# Split Data

In [5]:
x_train, x_test, y_train, y_test = (
    train_test_split(reviews, 
                     label, 
                     test_size=test_size_split, 
                     stratify = label, 
                     random_state=42)
)

print(f'Train\t| X shape: {x_train.shape}\tY shape: {y_train.shape}')
print(f'Test\t| X shape: {x_test.shape}\tY shape: {y_test.shape}')

Train	| X shape: (15522,)	Y shape: (15522, 3)
Test	| X shape: (1725,)	Y shape: (1725, 3)


# Feature Extraction

In [6]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features = MAX_FEATURES)
vectorizer.fit(x_train)

x_train_tfidf = vectorizer.fit_transform(x_train).todense()
x_test_tfidf = vectorizer.transform(x_test).todense()

# Feature Expansion

In [7]:
# Load trained fasttext model
model_exp = FastText.load('fasttext_news.model').wv

In [8]:
def feature_expansion(df,feature):
    for col in tqdm(df.columns): #loop per kolom
        try: 
            sim_word = model_exp.similar_by_word(col, topn = 20) #Mencari similarity berdasarkan nilai n
        except:
            sim_word = []
        if sim_word != []: #jika similarity tidak kosong
            for term in [sim_word[i][0] for i in range(len(sim_word))]: #loop per-word yang ada di Similarity
                if term in feature:
                    #untuk semua kolom yang mempunyai nilai 0 di kolom, tetapi mempunyai nilai yang bukan 0 pada kolom term
                    #nilainya diganti dengan nilai kolom term yang mempunyai nilai bukan 0
                    df[col][(df[col]==0) & (df[term]!=0)] = df[term][(df[col]==0) & (df[term]!=0)]
    return df

In [9]:
feature_ext = vectorizer.get_feature_names_out()

df_x_train = pd.DataFrame(x_train_tfidf, columns= feature_ext)
df_x_test = pd.DataFrame(x_test_tfidf, columns= feature_ext)

df_exp_x_train = feature_expansion(df_x_train, feature_ext)
df_exp_x_test = feature_expansion(df_x_test, feature_ext)

x_train = np.expand_dims(df_exp_x_train, 1)
x_test = np.expand_dims(df_exp_x_test, 1)

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [05:23<00:00, 30.88it/s]


# Model RNN

In [10]:
def RNN(x_train_c, y_train_c, x_test_c, y_test_c):
    rnn = Sequential()
    rnn.add(SimpleRNN(units=256, activation='relu', recurrent_dropout=0.2, return_sequences=True))
    rnn.add(Dropout(0.2))
    rnn.add(SimpleRNN(units=128, activation='relu', return_sequences=True))
    rnn.add(Dropout(0.2))
    rnn.add(SimpleRNN(units=64, activation='relu'))
    rnn.add(Dropout(0.2))
    rnn.add(Dense(units=3, activation='softmax'))
    
    rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    rnn.build(x_train_c.shape)
    
    early_stopping = EarlyStopping(monitor='val_loss', mode = 'min', verbose = 1, patience=3, min_delta=0.0001)
    model_checkpoint = ModelCheckpoint('rnn_tfidf_ft_plot.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)
    
    history = rnn.fit(x_train_c, y_train_c, epochs= 10, batch_size=128, 
                      validation_data=(x_test_c, y_test_c), verbose = 1,
                      callbacks=[early_stopping, model_checkpoint])
    
    model = load_model('rnn_tfidf_ft_plot.h5')
    y_pred = model.predict(x_test_c)
    y_pred_cat = y_pred.argmax(axis=1)
    y_test_cat = np.argmax(y_test_c, axis=1)
   
    cm = confusion_matrix(y_test_cat, y_pred_cat)
    classreport = classification_report(y_test_cat, y_pred_cat)
    f1 = f1_score(y_test_cat, y_pred_cat,  average="macro")
    recall = recall_score(y_test_cat, y_pred_cat, average="macro")
    precision = precision_score(y_test_cat, y_pred_cat, average="macro")
    accuracy = accuracy_score(y_test_cat, y_pred_cat)

    print(classreport)
    print("F1 Score : ", f1)
    print("Precision : ", precision)
    print("Recall : ", recall)
    print("Accuracy : ", accuracy)

    return [f1, precision, recall, accuracy, cm]

# Running

In [11]:
hasil = []

for i in range(1, 6):
    hasil.append([str(i)] + RNN(x_train, y_train, x_test, y_test))

Epoch 1/10
121/122 [============================>.] - ETA: 0s - loss: 0.8941 - accuracy: 0.5845
Epoch 1: val_accuracy improved from -inf to 0.66261, saving model to rnn_tfidf_ft_plot.h5
122/122 [==============================] - 21s 66ms/step - loss: 0.8938 - accuracy: 0.5848 - val_loss: 0.7853 - val_accuracy: 0.6626
Epoch 2/10
122/122 [==============================] - ETA: 0s - loss: 0.7327 - accuracy: 0.6873
Epoch 2: val_accuracy improved from 0.66261 to 0.68232, saving model to rnn_tfidf_ft_plot.h5
122/122 [==============================] - 6s 52ms/step - loss: 0.7327 - accuracy: 0.6873 - val_loss: 0.7423 - val_accuracy: 0.6823
Epoch 3/10
121/122 [============================>.] - ETA: 0s - loss: 0.6502 - accuracy: 0.7298
Epoch 3: val_accuracy did not improve from 0.68232
122/122 [==============================] - 5s 41ms/step - loss: 0.6499 - accuracy: 0.7300 - val_loss: 0.7773 - val_accuracy: 0.6690
Epoch 4/10
121/122 [============================>.] - ETA: 0s - loss: 0.5733 - ac

Epoch 5/10
121/122 [============================>.] - ETA: 0s - loss: 0.4978 - accuracy: 0.8000
Epoch 5: val_accuracy improved from 0.67768 to 0.68464, saving model to rnn_tfidf_ft_plot.h5
122/122 [==============================] - 3s 27ms/step - loss: 0.4978 - accuracy: 0.7999 - val_loss: 0.8169 - val_accuracy: 0.6846
Epoch 6/10
120/122 [============================>.] - ETA: 0s - loss: 0.4192 - accuracy: 0.8396
Epoch 6: val_accuracy did not improve from 0.68464
122/122 [==============================] - 3s 28ms/step - loss: 0.4189 - accuracy: 0.8397 - val_loss: 0.8513 - val_accuracy: 0.6765
Epoch 6: early stopping
54/54 [==============================] - 1s 6ms/step
              precision    recall  f1-score   support

           0       0.60      0.62      0.61       368
           1       0.74      0.68      0.71       657
           2       0.69      0.73      0.71       700

    accuracy                           0.68      1725
   macro avg       0.67      0.67      0.67      17

# Save Output

In [12]:
df = pd.DataFrame(hasil, columns =['iterasi', 'f1', 'precision', 'recall', 'accuracy', 'cm'])
#df.to_excel('rnn_tfidf_ft_plot.xlsx')
df.head()

,iterasi,f1,precision,recall,accuracy,cm
0,1,0.656354,0.684319,0.646900,0.682319,"[[167, 89, 112], [40, 471, 146], [35, 126, 539]]"
1,2,0.678939,0.682495,0.676301,0.694493,"[[211, 70, 87], [54, 487, 116], [73, 127, 500]]"
2,3,0.661636,0.675059,0.655264,0.685217,"[[181, 90, 97], [44, 470, 143], [61, 108, 531]]"
3,4,0.673256,0.673581,0.674200,0.684638,"[[228, 60, 80], [62, 445, 150], [92, 100, 508]]"
4,5,0.669067,0.680282,0.662909,0.688696,"[[192, 85, 91], [44, 472, 141], [62, 114, 524]]"


In [13]:
print("Accuracy : %.2f" % (df['accuracy'].mean()*100))
print("F1-Score : %.2f" % (df['f1'].mean()*100))

Accuracy : 68.71
F1-Score : 66.79
